<a href="https://colab.research.google.com/github/KaveeshBaddage/DataScienceImpl/blob/main/Copy_of_Generative_Adversarial_Network_(GAN)_for_MNIST_kaveesha_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Load MNIST Data set to Run Time



transform=ToTensor() <- converts data in the range 0-255 to 0-1.



In [2]:
transform = transforms.Compose([transforms.ToTensor(),
  transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(
    root='data/',
    train=True,
    download=True,
    transform=transform
    )  
print("train_dataset.data.size() ->",train_dataset.data.size())

# Dataset stores the samples and their corresponding labels, and DataLoader wraps an iterable around the Dataset to enable easy access to the samples.
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, 
    batch_size=100,
    shuffle=True
    # num_workers=0
    )

test_dataset = datasets.MNIST(
    root='data/', 
    train=False, 
    transform=transform
    )
print("test_dataset_sizec ->",test_dataset.data.size())


test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, 
    batch_size=100,
    # num_workers=0
    shuffle=True
    )


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw




Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw




Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw




Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw

train_dataset.data.size() -> torch.Size([60000, 28, 28])
test_dataset_sizec -> torch.Size([10000, 28, 28])


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# torch.utils.data.DataLoader Parameters

dataset (Dataset) – dataset from which to load the data.

batch_size (int, optional) – how many samples per batch to load (default: 1).

num_workers (int, optional) – how many subprocesses to use for data loading. 0 means that the data will be loaded in the main process.


----------

The iter() function creates an object which can be iterated one element at a time.



In [3]:
# defining Discriminator class

class Discriminator(nn.Module):

    def __init__(self, input_size, d_output_size):
        # access methods from parent class nn.Module
        super(Discriminator, self).__init__()
        
        # define hidden linear layers
        self.fc1 = nn.Linear(input_size, 1024)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features//2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc1.out_features//2)
        
        # final fully-connected layer
        self.fc4 = nn.Linear(self.fc3.out_features, d_output_size)
        
    def forward(self, x):
        # flatten image
        x = x.view(-1, 28*28) #  reshape the tensor to 28*28 tensor
        # all hidden layers
        x = F.leaky_relu(self.fc1(x), 0.2) # (input, negative_slope=0.2) This function returns x if it receives any positive input, but for any negative value of x, it returns a really small value which is 0.02 times x
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.dropout(x, 0.3)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = F.dropout(x, 0.3)
        # final layer
        out = torch.sigmoid(self.fc4(x))


        return out

In [4]:
# defining generator class 1

class Generator(nn.Module):

    def __init__(self, input_size, g_output_size):
        super(Generator, self).__init__()
        
        # define hidden linear layers
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features*2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc1.out_features*2)
        
        # final fully-connected layer
        self.fc4 = nn.Linear(self.fc3.out_features, g_output_size)

    def forward(self, x):
        # all hidden layers
        x = F.leaky_relu(self.fc1(x), 0.2) # (input, negative_slope=0.2)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.leaky_relu(self.fc3(x), 0.2)
        out = torch.tanh(self.fc4(x))

        return out

Reference - https://deeplizard.com/learn/video/bH9Nkg7G8S0

nn.Sequential - A sequential container. Modules will be added to it in the order they are passed in the constructor
The Sequential class allows us to build PyTorch neural networks on-the-fly without having to build an explicit class

---------

Rectified Linear Activation Function(ReLU) do

if(input > 0){
	return input 
  }
else{
	return 0
}

In [5]:

# Discriminator hyperparams

# Size of input image to discriminator (28*28)
input_size = 784
# Size of discriminator output (real or fake)
d_output_size = 1
# Size of last hidden layer in the discriminator
d_hidden_size = 64

# Generator hyperparams

# Size of latent vector to give to generator
z_size = 100
# Size of discriminator output (generated image)
g_output_size = 784
# Size of first hidden layer in the generator
g_hidden_size = 32


D = Discriminator(input_size, d_output_size).to(device)
G = Generator(z_size, g_output_size).to(device)

# check that they are as you expect
print(D)
print()
print(G)

Discriminator(
  (fc1): Linear(in_features=784, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=1, bias=True)
)

Generator(
  (fc1): Linear(in_features=100, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=784, bias=True)
)


In [6]:
# loss
criterion = nn.BCELoss() 

# optimizer
lr = 0.0002 
G_optimizer = optim.Adam(G.parameters(), lr = lr)
D_optimizer = optim.Adam(D.parameters(), lr = lr)

In [7]:
def D_train(x):
    #=======================Train the discriminator=======================#
    D.zero_grad()

    # train discriminator on real
    x_real, y_real = x.view(-1, 28*28), torch.ones(100, 1)
    x_real, y_real = Variable(x_real.to(device)), Variable(y_real.to(device))

    D_output = D(x_real)
    D_real_loss = criterion(D_output, y_real)
    D_real_score = D_output

    # train discriminator on facke
    z = Variable(torch.randn(100, 100).to(device))
    x_fake, y_fake = G(z), Variable(torch.zeros(100, 1).to(device))

    D_output = D(x_fake)
    D_fake_loss = criterion(D_output, y_fake)
    D_fake_score = D_output

    # gradient backprop & optimize ONLY D's parameters
    D_loss = D_real_loss + D_fake_loss
    D_loss.backward()
    D_optimizer.step()
        
    return  D_loss.data.item()

In [8]:
def G_train(x):
    #=======================Train the generator=======================#
    G.zero_grad()

    z = Variable(torch.randn(100, 100).to(device))
    y = Variable(torch.ones(100, 1).to(device))

    G_output = G(z)
    D_output = D(G_output)
    G_loss = criterion(D_output, y)

    # gradient backprop & optimize ONLY G's parameters
    G_loss.backward()
    G_optimizer.step()
        
    return G_loss.data.item()

*    train_loader contains 938 batches and each batch has 64 tensors. Because we define batch_size as 64 when storing dataset in to the DataLoader.
    *    938 X 64 = 60032  // which equals to train_dataset:60000 
*    Each tensor in a batch represent single image. 
*    Image representing tensor  has following format.
    *    `[[28 real numbers][28 real numbers]]`



In [9]:
n_epoch = 200
for epoch in range(1, n_epoch+1):           
    D_losses, G_losses = [], []
    for batch_idx, (x, _) in enumerate(train_loader):
        D_losses.append(D_train(x))
        G_losses.append(G_train(x))

    print('[%d/%d]: loss_d: %.3f, loss_g: %.3f' % (
            (epoch), n_epoch, torch.mean(torch.FloatTensor(D_losses)), torch.mean(torch.FloatTensor(G_losses))))

[1/200]: loss_d: 1.026, loss_g: 2.007
[2/200]: loss_d: 1.039, loss_g: 2.352
[3/200]: loss_d: 0.882, loss_g: 2.287
[4/200]: loss_d: 0.408, loss_g: 3.056
[5/200]: loss_d: 0.277, loss_g: 3.703
[6/200]: loss_d: 0.373, loss_g: 3.410
[7/200]: loss_d: 0.390, loss_g: 3.705
[8/200]: loss_d: 0.451, loss_g: 3.050
[9/200]: loss_d: 0.503, loss_g: 3.042
[10/200]: loss_d: 0.518, loss_g: 2.815
[11/200]: loss_d: 0.574, loss_g: 2.562
[12/200]: loss_d: 0.650, loss_g: 2.313
[13/200]: loss_d: 0.716, loss_g: 2.028
[14/200]: loss_d: 0.723, loss_g: 2.018
[15/200]: loss_d: 0.815, loss_g: 1.835
[16/200]: loss_d: 0.760, loss_g: 1.932
[17/200]: loss_d: 0.784, loss_g: 1.951
[18/200]: loss_d: 0.760, loss_g: 1.996
[19/200]: loss_d: 0.739, loss_g: 2.045
[20/200]: loss_d: 0.806, loss_g: 1.847
[21/200]: loss_d: 0.823, loss_g: 1.804
[22/200]: loss_d: 0.807, loss_g: 1.841
[23/200]: loss_d: 0.853, loss_g: 1.719
[24/200]: loss_d: 0.890, loss_g: 1.659
[25/200]: loss_d: 0.901, loss_g: 1.628
[26/200]: loss_d: 0.867, loss_g: 1

Enumerate() method adds a counter to an iterable and returns it in a form of enumerate objec

In [21]:
import pickle as pkl

# Save the Generator
with open('G.pkl', 'wb') as f:
    pkl.dump(G, f)

# Save the Discriminator
with open('D.pkl', 'wb') as f:
    pkl.dump(D, f)

In [ ]:
with open('train_samples.pkl', 'rb') as f:
    samples = pkl.load(f)

In [19]:
# from torch.autograd import Variable
# #device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
# bs = 100 #Batch Size
# z_dim = 100 #Z dimensions
# z = Variable(torch.randn(bs, z_dim).to(device))

from torchvision.utils import save_image
import numpy as np


#create 100 latent vectors
sample_size=100
z = np.random.uniform(-1, 1, size=(sample_size, z_size))
z = torch.from_numpy(z).float()

print(z[0].size())

G.eval() # eval mode

root = '/content'


for i, image_tensor in enumerate(z):
    image_tensor = image_tensor.detach()
    print(i,image_tensor)

    # save latent vector related to the image
    f = open(root + r'/images/%d.txt' %i,'a')
    f.write(np.array2string(image_tensor.numpy(), separator=','))
    f.close()

    # create image using latent vector
    # G.eval()
    rand_image = G(image_tensor.to(device)) 
    #view_samples(0, [rand_images])
    #print(rand_image.size())

    # save image
    rand_image = rand_image.view(-1, 28) # convert [784] tensor in to [28,28] tensor to create image
    print(rand_image.size())
    save_image(rand_image, root + r'/images/%d.png' %i, normalize = True)


#create a zip file contains generated image files and Latent samples
import shutil
shutil.make_archive('images', 'zip', 'images')

torch.Size([100])
0 tensor([ 0.8556,  0.4074, -0.0218, -0.3973, -0.6866, -0.6197,  0.7324, -0.4372,
        -0.1254,  0.8387,  0.9840,  0.9284, -0.5886, -0.2404, -0.5143, -0.2858,
         0.2371, -0.4996, -0.4330,  0.9834, -0.8887,  0.9717, -0.9183, -0.3658,
         0.7402,  0.4975,  0.4144, -0.9301,  0.9612, -0.6616, -0.4141, -0.2613,
        -0.2746,  0.3475,  0.6944,  0.3345,  0.7300, -0.3056, -0.8632,  0.7307,
         0.9098, -0.9716,  0.6017, -0.9177, -0.1903,  0.7268, -0.4842, -0.0358,
         0.7303,  0.5817,  0.2215,  0.1579,  0.0793, -0.0170, -0.2644,  0.2550,
        -0.9804, -0.8499, -0.9322, -0.8484,  0.7796,  0.8164, -0.2543, -0.4399,
         0.8188, -0.3773, -0.7651, -0.1895,  0.0302,  0.2814,  0.3444,  0.3435,
        -0.5848, -0.3228,  0.5790,  0.7154, -0.5570,  0.1131,  0.6380,  0.6618,
        -0.7611, -0.2241,  0.9998, -0.6761,  0.4445,  0.7477,  0.4846, -0.4237,
         0.5797,  0.5950, -0.1985, -0.4026,  0.2842, -0.2701,  0.5335,  0.6948,
         0.6438, -0.

'/content/images.zip'